In [2]:
import numpy as np
import pandas as pd
np.random.seed(0)

In [3]:
X=[[1,2,3,2.5],
   [2.0,5.0,-1.0,2.0],
   [-1.5,2.7,3.3,-0.8]]

In [61]:
class Layer_Dense:
    def __init__(self,n_input,n_neurons,learning_rate=0.1):
        self.weights=np.random.randn(n_input,n_neurons)*np.sqrt(1 / n_input)
        self.biases=np.zeros((1,n_neurons))
        self.learning_rate=0.01
    def forward(self,inputs):
        self.inputs=inputs
        self.output=np.dot(inputs,self.weights)+self.biases
    def backward(self, dvalues):
        # Gradients of weights and biases
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient for the previous layer
        self.dinputs = np.dot(dvalues, self.weights.T)
    def update(self):
        self.weights -= self.learning_rate * self.dweights
        self.biases -= self.learning_rate * self.dbiases
        
class Activation_ReLU:
    def forward(self,inputs):
        self.inputs=inputs
        self.output=np.maximum(0,inputs)
    def backward(self, dvalues):
        # Derivative of ReLU: If input > 0, pass the gradient, else set to 0
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0
        
class Activation_Softmax:
    def forward(self,inputs):
        self.inputs=inputs
        exp_values= np.exp(inputs-np.max(inputs,axis=1,keepdims=True))
        probs=exp_values/np.sum(exp_values,axis=1,keepdims=True)
        self.output=probs
        
        def backward(self, dvalues):
        # Initialize gradient array
            self.dinputs = np.empty_like(dvalues)

            for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flatten softmax output
                single_output = single_output.reshape(-1, 1)
            # Compute Jacobian matrix
                jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Compute gradient
                self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


class Loss:
    def calculate(self,output,y):
        sample_losses=self.forward(output,y)
        data_loss=np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self , y_pred,y_true):
        samples=len(y_pred)
        y_pred_clipped=np.clip(y_pred,1e-7,1-1e-7)
        if len(y_true.shape)==1:
            correct_confidences=y_pred_clipped[range(samples),y_true]
        elif len(y_true.shape)==2:
            correct_confidences=np.sum(y_pred_clipped*y_true,axis=1)
        negative_log=-np.log(correct_confidences)
        return negative_log
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        # If labels are sparse (categorical), convert them to one-hot encoding
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Compute gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

        
class Softmax_CrossEntropy_Loss:
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        return self.loss.calculate(self.activation.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)

        # If labels are sparse, convert to one-hot encoding
        if len(y_true.shape) == 1:
            y_true = np.eye(dvalues.shape[1])[y_true]

        # Gradient calculation
        self.dinputs = dvalues - y_true
        self.dinputs = self.dinputs / samples  # Normalize

In [20]:
layer1 = Layer_Dense(3, 4)
activation1 = Activation_ReLU()
layer2 = Layer_Dense(4, 3)
activation2= Activation_ReLU()
layer3 = Layer_Dense(3, 3)
loss_activation = Softmax_CrossEntropy_Loss()

# Forward pass
X = np.array([[1, 2, 3], [0.5, -0.2, 0.1], [0.3, 0.8, -1.2]])  # Example input
y = np.array([0, 1, 2])  # Example labels


for i in range(3000):
    layer1.forward(X)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    activation2.forward(layer2.output)
    layer3.forward(activation2.output)
    loss= loss_activation.forward(layer3.output,y)
    loss_activation.backward(loss_activation.activation.output,y)
    layer3.backward(loss_activation.dinputs)
    activation2.backward(layer3.dinputs)
    layer2.backward(loss_activation.dinputs)
    activation1.backward(layer2.dinputs)
    layer1.backward(activation1.dinputs)
    layer1.update()
    layer2.update()
    layer3.update()
    if i % 100==0:
        print("At ",i," loss :",loss)

At  0  loss : 1.0986119163263455
At  100  loss : 1.0986124424221657
At  200  loss : 1.0986097220711775
At  300  loss : 1.0985542103385197
At  400  loss : 1.0963541635113236
At  500  loss : 1.0049304412047684
At  600  loss : 0.7384816938395264
At  700  loss : 0.6673135528113064
At  800  loss : 0.6259004648313631
At  900  loss : 0.5961774991734413
At  1000  loss : 0.5737256717095486
At  1100  loss : 0.5563365465871528
At  1200  loss : 0.5426577755630128
At  1300  loss : 0.531760092565169
At  1400  loss : 0.5229694374732644
At  1500  loss : 0.5156955376702689
At  1600  loss : 0.5094852443288578
At  1700  loss : 0.5038203752200124
At  1800  loss : 0.4977902145437512
At  1900  loss : 0.4894543292632106
At  2000  loss : 0.47451195279376307
At  2100  loss : 0.44484987914160606
At  2200  loss : 0.3910492057846083
At  2300  loss : 0.32497248851048194
At  2400  loss : 0.25811522251395824
At  2500  loss : 0.19889149420334015
At  2600  loss : 0.15220615984846853
At  2700  loss : 0.1176840211674729

In [44]:
data=pd.read_csv('mushroom_cleaned.csv')

In [14]:
import sklearn
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder



In [67]:
def load_data():
   
    train_dataset = h5py.File('archive (1)/catvnoncat/train_catvnoncat.h5', "r")
    test_dataset = h5py.File('archive (1)/catvnoncat/test_catvnoncat.h5', "r")

    train_x_orig = np.array(train_dataset["train_set_x"][:])  # Training images
    train_y = np.array(train_dataset["train_set_y"][:])  # Training labels

    test_x_orig = np.array(test_dataset["test_set_x"][:])  # Test images
    test_y = np.array(test_dataset["test_set_y"][:])  # Test labels

   # Reshape to (1, m_test)

    return train_x_orig, train_y, test_x_orig, test_y

In [68]:
train_x_orig, train_y, test_x_orig, test_y= load_data()

In [69]:
train_x = train_x_orig.reshape(train_x_orig.shape[0], -1) # The "-1" makes reshape flatten the remaining dimensions
test_x= test_x_orig.reshape(test_x_orig.shape[0], -1)
train_x=train_x/255
test_x=test_x/255
def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y.reshape(-1)]

num_classes = len(np.unique(train_y))
train_y = one_hot_encode(train_y, num_classes)
test_y = one_hot_encode(test_y, num_classes)
print(train_x.shape)
print(train_y.shape)

(209, 12288)
(209, 2)


In [70]:
layer1 = Layer_Dense(12288, 64)
activation1 = Activation_ReLU()
layer2 = Layer_Dense(64, 2)
loss_activation = Softmax_CrossEntropy_Loss()


for i in range(3000):
    layer1.forward(train_x)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    loss= loss_activation.forward(layer2.output,train_y)
    loss_activation.backward(loss_activation.activation.output,train_y)
    layer2.backward(loss_activation.dinputs)
    activation1.backward(layer2.dinputs)
    layer1.backward(activation1.dinputs)
    layer1.update()
    layer2.update()
    if i % 100==0:
        print("At ",i," loss :",loss)

At  0  loss : 0.8443547547258125
At  100  loss : 0.5090563027193618
At  200  loss : 0.42048667977579135
At  300  loss : 0.3299193279245668
At  400  loss : 0.23798135537098675
At  500  loss : 0.1643932646997144
At  600  loss : 0.10716146346522688
At  700  loss : 0.07870568999024516
At  800  loss : 0.055423935599405054
At  900  loss : 0.04098350521535681
At  1000  loss : 0.0327517561019273
At  1100  loss : 0.027012450250726728
At  1200  loss : 0.022821720295717763
At  1300  loss : 0.019572965512898832
At  1400  loss : 0.017023704533922355
At  1500  loss : 0.014990290542001345
At  1600  loss : 0.013346132688390877
At  1700  loss : 0.011983080733260838
At  1800  loss : 0.01084157355407832
At  1900  loss : 0.00987474446951633
At  2000  loss : 0.00905009280239437
At  2100  loss : 0.008336454500044149
At  2200  loss : 0.0077155222398116955
At  2300  loss : 0.007173746682750512
At  2400  loss : 0.006690704279983167
At  2500  loss : 0.006261437515269172
At  2600  loss : 0.005881982866175795
At 

In [71]:
layer1.forward(test_x)
activation1.forward(layer1.output)
layer2.forward(activation1.output)
loss_activation.forward(layer2.output,test_y)
predictions = np.argmax(loss_activation.activation.output, axis=1)
y_true_labels = np.argmax(test_y, axis=1)
accuracy = np.mean(predictions == y_true_labels)
print(accuracy)

0.76


In [44]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the Iris dataset
iris = load_iris()

# Create a DataFrame
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Features and target
X = df[iris.feature_names]
y = df['target']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

y_onehot = np.eye(len(np.unique(y)))[y]
# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y_onehot, test_size=0.3)

# Show the shapes of the resulting datasets
print(f"Training data shape: X_train: {X_train.shape}, y_train: {Y_train.shape}")
print(f"Test data shape: X_test: {X_test.shape}, y_test: {Y_test.shape}")


Training data shape: X_train: (105, 4), y_train: (105, 3)
Test data shape: X_test: (45, 4), y_test: (45, 3)


In [65]:
layer1 = Layer_Dense(4, 4)
activation1 = Activation_ReLU()
layer2 = Layer_Dense(4, 5)
activation2= Activation_ReLU()
layer3 = Layer_Dense(5, 3)
activation3= Activation_ReLU()
layer4 = Layer_Dense(3, 3)
loss_activation = Softmax_CrossEntropy_Loss()
for i in range(10000):
    layer1.forward(X_train)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    activation2.forward(layer2.output)
    layer3.forward(activation2.output)
    activation3.forward(layer3.output)
    layer4.forward(activation3.output)
    loss= loss_activation.forward(layer4.output,Y_train)
    loss_activation.backward(loss_activation.activation.output,Y_train)
    layer4.backward(loss_activation.dinputs)
    activation3.backward(layer4.dinputs)
    layer3.backward(activation3.dinputs)
    activation2.backward(layer3.dinputs)
    layer2.backward(activation2.dinputs)
    activation1.backward(layer2.dinputs)
    layer1.backward(activation1.dinputs)
    layer1.update()
    layer2.update()
    layer3.update()
    layer4.update()
    if i % 100==0:
        print("At ",i," loss :",loss)



At  0  loss : 1.0762847115622924
At  100  loss : 0.9370044543724079
At  200  loss : 0.7774894108843673
At  300  loss : 0.6733064844066425
At  400  loss : 0.6082622443353742
At  500  loss : 0.5653644289905496
At  600  loss : 0.5277611605660845
At  700  loss : 0.4949271551805345
At  800  loss : 0.4651064690727723
At  900  loss : 0.43566395115776885
At  1000  loss : 0.40645339373381945
At  1100  loss : 0.37716438065034236
At  1200  loss : 0.3481152260886858
At  1300  loss : 0.32002823869544583
At  1400  loss : 0.2934920019516602
At  1500  loss : 0.2689443697777041
At  1600  loss : 0.24628013897448747
At  1700  loss : 0.22593587862060582
At  1800  loss : 0.20768792302406364
At  1900  loss : 0.1914421954631651
At  2000  loss : 0.1772467427420168
At  2100  loss : 0.16475475400580547
At  2200  loss : 0.15376911299912857
At  2300  loss : 0.14417371754453165
At  2400  loss : 0.13574328195124527
At  2500  loss : 0.1282926269577896
At  2600  loss : 0.12164701656621052
At  2700  loss : 0.115685745

In [66]:
layer1.forward(X_train)
activation1.forward(layer1.output)
layer2.forward(activation1.output)
activation2.forward(layer2.output)
layer3.forward(activation2.output)
activation3.forward(layer3.output)
layer4.forward(activation3.output)
loss_activation.forward(layer4.output,Y_train)
predictions = np.argmax(loss_activation.activation.output, axis=1)
y_true_labels = np.argmax(Y_train, axis=1)
accuracy = np.mean(predictions == y_true_labels)
print(accuracy)

0.9904761904761905
